# This code uses both R and Pyhton within one single Jupyter notebook. 

Import necessary libraries

In [917]:
# Defaults:
import os
import pandas as pd
import numpy as np

# to load CSV data from URL
import io
import requests

# Graphics:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # For 3d plots. This import is necessary to have 3D plotting below
import pylab as pl
import seaborn as sns; sns.set()  # for plot styling

# R to Python libraries:
from rpy2.robjects import r
import rpy2.robjects.pandas2ri as pandas2ri
import rpy2.robjects as robjects
import rpy2.rinterface # import interface for having both R & Pyhton in  single framework

# R section:

In [2]:
# Load rpy2 in ipython:
%load_ext rpy2.ipython 

This part runs in R.
Reads data from the MPES website and saves them as RData.
The RData will then be converted to Pyhthon Pandas DataFrame

In [5]:
%%R

library(foreign)
library(survey)

url = "https://meps.ahrq.gov/mepsweb/data_files/pufs/"
file_path = "C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/"

list_data <- list("h181", "h178a" , "h178b", "h178d", "h178e", "h178f", "h178g", "h178h", "h178if1", "h180")

#list_data <- list("h178e", "h178f", "h178g", "h178h", "h178if1", "h180")

for (data_name in list_data)
{
file_url = paste(url, data_name, "ssp.zip",sep = "")
download.file(file_url, temp <- tempfile())

unzipped_file = unzip(temp)
name = read.xport(unzipped_file)
unlink(temp)  # Unlink to delete temporary file

file_address = paste(file_path, data_name,".Rdata",sep = "")
save(name, file=file_address)
rm(name)
}

# Back to Pyhton:

Read and convert the RData to Pandas DataFrame

In [4]:
# Define a function to read the RData and save it as CSV file:

def from_R_to_csv(file_path,list_data, data_type):
    for data_name in list_data:
        file_name = file_path+data_name+".Rdata"
        print(file_name)
        rf=robjects.r['load'](file_name)
        #acces file in env and convert
        name = 'name'
        df = pandas2ri.ri2py_dataframe(r[name])
        df.name = data_name
        df2 = pd.DataFrame()
        if data_type == 'FYC':
            df2 = df.loc[:,['DUPERSID','PERWT15F','VARSTR','VARPSU']]
        elif data_type == 'APPENDIX':
            df2 = df.loc[:,['DUPERSID', 'CONDIDX', 'EVNTIDX']]    
        elif data_type == 'MED COND':
            df2 = df.loc[:,['DUPERSID','CONDIDX','CCCODEX']]
        elif data_type == 'EVENTS':
            df2 = df      
        csv_file_name = file_path+data_name+".csv"
        df2.to_csv(csv_file_name)
        del df, df2

In [41]:
# To use the from_R_to_csv finction to read the saved RData for medical onditions and save them as CSV
path = "C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/"

# Event dictionary:
Event_dict =  {'h178a': 'Prescribed Medicines',
               'h178b': 'Dental Visits',
               'h178c': 'Other Medical Expenses',
               'h178d': 'Hospital Inpatient Stays',
               'h178e': 'Emergency Room Visits',
               'h178f': 'Outpatient Visits',
               'h178g': 'Office-Based Medical Provider Visits',
               'h178h': 'Home Health'}

# Assigning data:
FYC = ["h181"]
Events = ["h178a" , "h178b", "h178d", "h178e", "h178f", "h178g", "h178h"]
Event_Appendix  = ["h178if1"]
Med_Cond = ["h180"]

# Read and save data:
from_R_to_csv(path ,FYC, 'FYC')
from_R_to_csv(path ,Events, 'EVENTS')
from_R_to_csv(path ,Event_Appendix, 'APPENDIX')
from_R_to_csv(path ,Med_Cond, 'MED COND')

C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h181.Rdata
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h181.csv
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178a.Rdata
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178a.csv
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178b.Rdata
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178b.csv
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178d.Rdata
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/h178d.csv
C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/ME

In [95]:
# READ Appendix to MEPS Event (APX), Medical Condition (MEDC), and Full Year Consolidated (FYC) files 
#del MEDC,APX,FYC
path = "C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/"

APX_path = path + 'h178if1.csv'
APX = pd.read_csv(APX_path)

MEDC_path = path + 'h180.csv'
MEDC = pd.read_csv(MEDC_path)

FYC_path = path + 'h181.csv'
FYC = pd.read_csv(FYC_path)

In [175]:
# Merge APX & MEDC
#del df1, df3
df1 = pd.merge(MEDC, APX, on=['DUPERSID','CONDIDX'], how = 'inner')
print(df1.shape)
df3 = pd.merge(MEDC,APX,on=['DUPERSID','CONDIDX'], how = 'left')
print(df3.shape)
df4 = MEDC.join(APX, lsuffix='_MEDC', rsuffix='_APX')
print(df4.shape)
df5 = APX.join(MEDC, lsuffix='_APX', rsuffix='_MEDC')
print(df5.shape)

df3 = df3.astype(float)
df3['EVNTIDX'].replace('', np.nan, inplace=True)

df3[~df3.isin(['NaN', 'NaT']).any(axis=1)]
df7 = df3.dropna()
csv_file_name = path+"df7.csv"
print(csv_file_name)
df7.to_csv(csv_file_name)


(377736, 6)
(407985, 6)
(123227, 8)
(377736, 8)


C:\Users\ali\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


C:/Users/ali/Downloads/Data Science/Bootcamps/Data Incubator Fellowship/Final interview/MEPS_data/2015/df7.csv


In [176]:
# READ EVENT FILES

# Prescribed Medicines
RX_path = path + 'h178a.csv'
RX = pd.read_csv(RX_path)
RX = RX.rename(columns = {'LINKIDX':'EVNTIDX'})
print(RX.shape)

# Dental Visits
DVT_path = path + 'h178b.csv'
DVT = pd.read_csv(DVT_path)
DVT = DVT.rename(columns = {'LINKIDX':'EVNTIDX'})
print(DVT.shape)

# Hospital Inpatient Stays
IPT_path = path + 'h178d.csv'
IPT = pd.read_csv(IPT_path)
IPT = IPT.rename(columns = {'LINKIDX':'EVNTIDX'})
print(IPT.shape)

# Emergency Room Visits
ERT_path = path + 'h178e.csv'
ERT = pd.read_csv(ERT_path)
ERT = ERT.rename(columns = {'LINKIDX':'EVNTIDX'})
print(ERT.shape)

# Outpatient Visits
OPT_path = path + 'h178f.csv'
OPT = pd.read_csv(OPT_path)
OPT = OPT.rename(columns = {'LINKIDX':'EVNTIDX'})
print(OPT.shape)

# Office-Based Medical Provider Visits
OBV_path = path + 'h178g.csv'
OBV = pd.read_csv(OBV_path)
OBV = OBV.rename(columns = {'LINKIDX':'EVNTIDX'})
print(OBV.shape)

# Home Health
HHT_path = path + 'h178h.csv'
HHT = pd.read_csv(HHT_path)
HHT = HHT.rename(columns = {'LINKIDX':'EVNTIDX'})
print(HHT.shape)

(330453, 70)
(28090, 77)
(2921, 61)
(7163, 67)
(14178, 70)
(172388, 57)
(6663, 69)


CCCODEX: 
The condition categories used in the summary data tables were defined using Clinical Classification System (CCS) categories. The table below shows the CCS codes that make up each category.

In [179]:
df8 = pd.merge(RX,df7,on=['DUPERSID','EVNTIDX'], how = 'left')
print(df8.shape)

print(RX.shape)
print(df7.shape)
df9 = df8.dropna()
csv_file_name = path+"df9.csv"
print(csv_file_name)
df9.to_csv(csv_file_name)

mask = (df4.CCCODEX >=11) & (df4.CCCODEX <=45)
df5 = df4.loc[mask]

(358287, 74)
